# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9fe844e070>
├── 'a' --> tensor([[-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887]])
└── 'x' --> <FastTreeValue 0x7f9fe83a51f0>
    └── 'c' --> tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                        [ 1.1446, -0.3739, -0.7056, -3.0979],
                        [-0.7701, -0.2804,  0.3893,  0.9793]])

In [4]:
t.a

tensor([[-1.7311,  0.2431,  1.6695],
        [ 0.5082,  0.0604, -0.8887]])

In [5]:
%timeit t.a

65.1 ns ± 0.0331 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9fe844e070>
├── 'a' --> tensor([[-0.2117, -0.3795,  0.4265],
│                   [-1.0136,  0.3527, -0.6198]])
└── 'x' --> <FastTreeValue 0x7f9fe83a51f0>
    └── 'c' --> tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                        [ 1.1446, -0.3739, -0.7056, -3.0979],
                        [-0.7701, -0.2804,  0.3893,  0.9793]])

In [7]:
%timeit t.a = new_value

68.1 ns ± 0.0386 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887]]),
    x: Batch(
           c: tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                      [ 1.1446, -0.3739, -0.7056, -3.0979],
                      [-0.7701, -0.2804,  0.3893,  0.9793]]),
       ),
)

In [10]:
b.a

tensor([[-1.7311,  0.2431,  1.6695],
        [ 0.5082,  0.0604, -0.8887]])

In [11]:
%timeit b.a

64.2 ns ± 0.0295 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[0.3471, 1.0012, 0.1091],
               [1.4917, 1.2688, 0.4042]]),
    x: Batch(
           c: tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                      [ 1.1446, -0.3739, -0.7056, -3.0979],
                      [-0.7701, -0.2804,  0.3893,  0.9793]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.0829 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

949 ns ± 31 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 539 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9f306941f0>
├── 'a' --> tensor([[[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]],
│           
│                   [[-1.7311,  0.2431,  1.6695],
│                    [ 0.5082,  0.0604, -0.8887]]])
└── 'x' --> <FastTreeValue 0x7f9f306943a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 95.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9fbc837be0>
├── 'a' --> tensor([[-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887],
│                   [-1.7311,  0.2431,  1.6695],
│                   [ 0.5082,  0.0604, -0.8887]])
└── 'x' --> <FastTreeValue 0x7f9fbc837c10>
    └── 'c' --> tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                        [ 1.1446, -0.3739, -0.7056, -3.0979],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 46.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 78 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]],
       
               [[-1.7311,  0.2431,  1.6695],
                [ 0.5082,  0.0604, -0.8887]]]),
    x: Batch(
           c: tensor([[[-1.4322, -0.8014, -0.1613, -1.8809],
                       [ 1.1446, -0.3739, -0.7056, -3.0979],
                       [-0.7701, -0.2804,  0.3893,  0.9793]],
         

In [26]:
%timeit Batch.stack(batches)

79.5 µs ± 164 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887],
               [-1.7311,  0.2431,  1.6695],
               [ 0.5082,  0.0604, -0.8887]]),
    x: Batch(
           c: tensor([[-1.4322, -0.8014, -0.1613, -1.8809],
                      [ 1.1446, -0.3739, -0.7056, -3.0979],
                      [-0.7701, -0.2804,  0.3893,  0.9793],
                      [-1.4322, -0.8014, -0.1613, -1.8809],
                      [ 1.1446, -

In [28]:
%timeit Batch.cat(batches)

143 µs ± 435 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
